### Import les librairies

In [214]:
import numpy as np
import pandas as pd


### Importer les csv

In [215]:
path = './Data_use_case_Yelp/use_case/'
# path_sample = path + 'sample'

review_file = 'avis.csv'
categorie_file = 'categories.csv'
checkin_file = 'checkin.csv'
conseil_file = 'conseils.csv'
horaires_file = 'horaires.csv'
restaurant_features_file = 'restaurants_features.csv'
restaurant_file = 'restaurants.csv'
services_file = 'services.csv'
utilisateurs_file = 'utilisateurs.csv'

### Charger les données dans le dataframes

In [216]:
services = pd.read_csv(path + services_file, skip_blank_lines=True)
categories = pd.read_csv(path + categorie_file, skip_blank_lines=True)
horaires = pd.read_csv(path + horaires_file, skip_blank_lines=True)
business = pd.read_csv(path + restaurant_file, skip_blank_lines=True)
tip = pd.read_csv(path + conseil_file, skip_blank_lines=True)
checkins = pd.read_csv(path + checkin_file, skip_blank_lines=True)
user = pd.read_csv(path + utilisateurs_file, skip_blank_lines=True)
reviews = pd.read_csv(path + review_file, skip_blank_lines=True)
restaurant_features = pd.read_csv(path + restaurant_features_file, skip_blank_lines=True)

### Donner une moyenne d'étoiles par restaurants

In [217]:
avg_stars = reviews.groupby('restaurant_id')['etoiles'].mean().reset_index(name='avg_stars')

### Récupérer le nombres d'avis par restaurants

In [218]:
review_count_total = reviews.groupby('restaurant_id')['etoiles'].count().reset_index(name='review_count_total')

### Récupérer ratio d'avis favorables (étoile > 4)

In [219]:
positive_review_count = reviews[reviews['etoiles'] >= 4].groupby('restaurant_id')['avis_id'].count().reset_index(name='positif_review_count')

### Nombres totales de check-ins

In [220]:
checkins_total = checkins.groupby('restaurant_id')['date'].count().reset_index(name='checkins_total')

### Indiquer si le restaurant fait parti d'une chaine

In [221]:
business['is_chain'] = business.groupby('nom')['restaurant_id'].transform('count') >= 3

### cout moyen du restautant

In [222]:
prices = services[['restaurant_id', 'prix']].rename(columns={'prix': 'price_range'})

### Nombre d'utilisateur élite ayant commenté le restaurant

In [223]:
reviews_by_elites = reviews.merge(user[['utilisateur_id', 'elite']], on='utilisateur_id', how='left')

elite_review = reviews_by_elites[reviews_by_elites['elite'] == 1]

elite_users_count = elite_review.groupby('restaurant_id')['utilisateur_id'].nunique().reset_index(name='elite_user_count')

### : Moyenne des heures d’ouverture par jour

In [ ]:
from datetime import datetime



#### : spliter les horaires d'ouverture ( ouvreture : 0:00 fermeture : 23:59 et les transforme en heures)
def duration_opening_hours(time):

    ### valdier le format
    if pd.isna(time) or time == '0:0-0:0':
        return 0
    
    opening, closing = time.split('-')
    opening = datetime.strptime(opening, '%H:%M')
    closing = datetime.strptime(closing, '%H:%M')

    opening_hours = (closing - opening).seconds / 3600
    return opening_hours


### Exraire les jours de la semaines
days = [col for col in horaires.columns if col != 'restaurant_id']

### apply la fonction pour split l'horaire à chaque jour de la semaine
for day in days:
    horaires[day] = horaires[day].apply(duration_opening_hours)

### Calcul de la moyenne des heures d'ouverture par jour
horaires['avg_opening_hours'] = horaires[days].mean(axis=1).round(2)

avg_opening_hours = horaires[['restaurant_id', 'avg_opening_hours']]



    

### Rassembler le tout dans un dataframe 

In [225]:

dataset = business[['restaurant_id', 'nom', 'ville']].merge(avg_stars, on='restaurant_id', how='left') \
    .merge(review_count_total, on='restaurant_id', how='left') \
    .merge(positive_review_count, on='restaurant_id', how='left') \
    .merge(checkins_total, on='restaurant_id', how='left') \
    .merge(elite_users_count, on='restaurant_id', how='left') \
    .merge(avg_opening_hours, on='restaurant_id', how='left') \
    .merge(business[['restaurant_id', 'is_chain']], on='restaurant_id', how='left') \
    .merge(prices, on='restaurant_id', how='left')

### Rajouter le dataset dans le fichier restaurant_features_file.csv

In [226]:
### formatage 
dataset['avg_stars'] = dataset['avg_stars'].round(2)
dataset['positif_review_count'] = dataset['positif_review_count'].fillna(0).astype(int)
dataset['elite_user_count'] = dataset['elite_user_count'].fillna(0).astype(int)
dataset['checkins_total'] = dataset['checkins_total'].fillna(0).astype(int)

dataset.to_csv(path + restaurant_features_file, index=False)

In [227]:
restaurant_features

,restaurant_id,nom,ville,avg_stars,review_count_total,positif_review_count,checkins_total,elite_user_count,avg_opening_hours,is_chain,price_range
0,lCwqJWMxvIUQt1Re_tDn4w,Denny's,Las Vegas,2.40,72,22,181,6,0.00,True,2
1,pd0v6sOqpLhFJ7mkpIaixw,Ike's Love & Sandwiches,Phoenix,4.19,108,82,492,23,9.14,False,2
2,0vhi__HtC2L4-vScgDFdFw,Midori Japanese Cafe,Calgary,3.67,49,33,157,26,9.64,False,2
3,t65yfB9v9fqlhAkLnnUXdg,Pho U,Toronto,3.56,36,21,18,3,8.14,False,1
4,i7_JPit-2kAbtRTLkic2jA,John & Sons Oyster House,Toronto,3.85,88,63,110,36,7.93,False,3
...,...,...,...,...,...,...,...,...,...,...,...
34263,cjZfgcQwA6KmQ_ANWKN2aw,Bruegger's Bagels,McMurray,3.67,6,4,37,1,8.00,True,1
34264,Hq2edcOTjse7wjK2CwBijQ,Bistro Pointe-Claire,Pointe-Claire,3.64,11,6,5,0,9.79,False,2
34265,7KlpgRjjAmVabPzxcExs0g,Taco Mex,Phoenix,4.00,11,8,32,1,0.00,False,1
34266,0fY-zYyP2fDmp2YXFsuNTg,Gotham Provisions Company,Sun Prairie,4.11,18,13,7,1,5.00,False,1
